In [1]:
# Add ldcpy root to system path
import sys
sys.path.insert(0,'../../../')
                    
# Import ldcpy package
# Autoreloads package everytime the package is called, so changes to code will be reflected in the notebook if the above sys.path.insert(...) line is uncommented.
%load_ext autoreload
%autoreload 2
import ldcpy

#display the plots in this notebook
%matplotlib inline

In [2]:
# ts_col contains TS data
ts_col = ldcpy.open_datasets(["TS"],['../../../data/cam-fv/orig.TS.100days.nc', 
                                 '../../../data/cam-fv/zfp1.0.TS.100days.nc', 
                                 '../../../data/cam-fv/zfp1e-1.TS.100days.nc',
                                    '../../../data/cam-fv/zfp1e-3.TS.100days.nc'],
                         ['orig', 'zfp1.0', 'zfp1e-1', 'zfp1e-3'])
# prect_col contains PRECT data
prect_col = ldcpy.open_datasets(["PRECT"],['../../../data/cam-fv/orig.PRECT.60days.nc', 
                                     '../../../data/cam-fv/zfp1e-7.PRECT.60days.nc', 
                                    '../../../data/cam-fv/zfp1e-9.PRECT.60days.nc', 
                                     '../../../data/cam-fv/zfp1e-11.PRECT.60days.nc'],
                         ['orig', 'zfp1e-7', 'zfp1e-9', 'zfp1e-11'])
# col_ds3 contains 3D T data (here we specify the chunk to be a single timeslice)
t_col = ldcpy.open_datasets(
    ["T"],
    [
        "../../../data/cam-fv/cam-fv.T.3months.nc",
        "../../../data/cam-fv/c.fpzip.cam-fv.T.3months.nc",
    ],
    ["orig", "comp"]
)
precsc_col = ldcpy.open_datasets(["PRECSC"], 
               ["/Users/abaker/alli/scratch/precsc.orig.nc","/Users/abaker/alli/scratch/t10.b.e11.BRCP85C5CNBDRD.f09_g16.031.cam.h1.PRECSC.20060101-20801231.nc"
               ], ["orig", "comp"])


dataset size in GB 0.09

dataset size in GB 0.05

dataset size in GB 0.04

dataset size in GB 0.00



In [4]:
tso = ts_col.TS.sel(collection="orig").isel(time=1)
tsz = ts_col.TS.sel(collection="zfp1.0").isel(time=1)
so_d = tso.data.compute()
tsz_d = tsz.data.compute()

In [337]:
from skimage.metrics import structural_similarity as ssim
maxr = max(tso_d.max(), tsz_d.max())
minr = min(tso_d.min(), tsz_d.min())
myrange = maxr-minr
sog = ssim(tso_d, tsz_d, K1=1e-15, K2 = 1e-15, multichannel=False, data_range=myrange, gaussian_weights=True,use_sample_covariance=False)
sog

0.8748169889982416

In [307]:
from skimage.metrics import structural_similarity as ssim
maxr = max(tso_d.max(), tsz_d.max())
minr = min(tso_d.min(), tsz_d.min())
myrange = maxr-minr
sc_o = (tso_d - minr)/myrange
sc_z = (tsz_d - minr)/myrange
sog = ssim(sc_o, sc_z, K1=1e-15, K2 = 1e-15, multichannel=False, data_range=1.0, gaussian_weights=True,use_sample_covariance=False)
sog

0.9956937317280954

In [343]:

dm = ldcpy.metrics.DiffMetrics(tso.compute(), tsz.compute(), aggregate_dims=['lat', 'lon'])
ssim_fp = dm.get_diff_metric('ssim_fp')
ssim_o = dm.get_diff_metric('ssim')
ssim_fp_old = dm.get_diff_metric('ssim_fp_old')

print(ssim_fp)
print(ssim_o)
print(ssim_fp_old)

0.9809314180354818
0.9984370498556349
0.9996643775728832


In [324]:
from skimage.metrics import structural_similarity as ssim
import skimage
to = skimage.img_as_ubyte(sc_o)/255
tz = skimage.img_as_ubyte(sc_z)/255

test = ssim(to, tz, K1=1e-15, K2 = 1e-15, multichannel=False, data_range=1.0, gaussian_weights=True,use_sample_covariance=False)
test

0.981803814603496

In [311]:
to = sc_o*255
tz = sc_z*255
ro = np.round(to)/255
rz = np.round(tz)/255
test = ssim(to, tz, K1=1e-15, K2 = 1e-15, multichannel=False, data_range=1.0, gaussian_weights=True,use_sample_covariance=False)
test

0.9956937334057131

In [346]:
po = precsc_col.PRECSC.sel(collection="orig").isel(time=0)
pz = precsc_col.PRECSC.sel(collection="comp").isel(time=0)
po_d = po.data.compute()
pz_d = pz.data.compute()

In [347]:
dm = ldcpy.metrics.DiffMetrics(po.compute(), pz.compute(), aggregate_dims=['lat', 'lon'])
ssim_fp = dm.get_diff_metric('ssim_fp')
ssim = dm.get_diff_metric('ssim')
ssim_fp_old = dm.get_diff_metric('ssim_fp_old')
print(ssim_fp)
print(ssim)
print(ssim_fp_old)

0.9999767059347989
0.9999999595814616
0.9999999741322456


In [314]:
from skimage.metrics import structural_similarity as ssim
maxr = max(po_d.max(), pz_d.max())
minr = min(po_d.min(), pz_d.min())
myrange = maxr-minr
sc_po = (po_d - minr)/myrange
sc_pz = (pz_d - minr)/myrange
sog = ssim(sc_po, sc_pz, K1=1e-15, K2 = 1e-15, multichannel=False, data_range=1.0, gaussian_weights=True,use_sample_covariance=False)
sog

0.9919888401297736

In [315]:
tpo = sc_po*255
tpz = sc_pz*255
rpo = np.round(tpo)/255
rpz = np.round(tpz)/255
test = ssim(rpo, rpz, K1=1e-15, K2 = 1e-15, multichannel=False, data_range=1.0, gaussian_weights=True,use_sample_covariance=False)
test



0.9999745202678167

0.9999745202674839

In [266]:
ds1 = precsc_col.isel(time=1)
ldcpy.compare_stats(ds1, "PRECSC", "orig","comp", significant_digits=4)

 
mean orig                          : 3.207e-11
mean comp                          : 3.208e-11
mean diff                          : -1.255e-14
 
variance orig                      : 2.427e-19
variance comp                      : 2.427e-19
 
standard deviation orig            : 4.926e-10
standard deviation comp            : 4.927e-10
 
max value orig                     : 3.064e-08
max value comp                     : 3.059e-08
min value orig                     : 0
min value comp                     : -2.91e-11
 
max abs diff                       : 5.242e-11
min abs diff                       : 0
mean abs diff                      : 4.477e-14
mean squared diff                  : 1.574e-28
root mean squared diff             : 7.266e-13
normalized root mean squared diff  : 2.371e-05
normalized max pointwise error     : 0.001711
pearson correlation coefficient    : 1
ks p-value                         : 0.1259
spatial relative error(% > 0.0001) : 2.897
max spatial relative error        

In [267]:
ds0 = precsc_col.isel(time=0)
ldcpy.compare_stats(ds0, "PRECSC", "orig","comp", significant_digits=8)

 
mean orig                          : 1.9429568e-11
mean comp                          : 1.9437816e-11
mean diff                          : -8.2476373e-15
 
variance orig                      : 3.6298469e-19
variance comp                      : 3.6294346e-19
 
standard deviation orig            : 6.0248755e-10
standard deviation comp            : 6.0245333e-10
 
max value orig                     : 6.0717383e-08
max value comp                     : 6.0594175e-08
min value orig                     : 0
min value comp                     : -5.8207661e-11
 
max abs diff                       : 1.2320811e-10
min abs diff                       : 0
mean abs diff                      : 3.8053669e-14
mean squared diff                  : 6.8023521e-29
root mean squared diff             : 1.0342062e-12
normalized root mean squared diff  : 1.7033115e-05
normalized max pointwise error     : 0.0020292065
pearson correlation coefficient    : 0.99999853
ks p-value                         : 0.9999991


In [375]:
p_orig = precsc_col["PRECSC"].sel(collection="orig").isel(time=1).compute()
p_zfp = precsc_col["PRECSC"].sel(collection="comp").isel(time=1).compute()

dm = ldcpy.metrics.DiffMetrics(p_orig, p_zfp, aggregate_dims=['lat', 'lon'])
#ssim = dm.get_diff_metric('ssim')
ssim_fp = dm.get_diff_metric('ssim_fp')
ssim_fp_old = dm.get_diff_metric('ssim_fp_old')


In [376]:
print('ssim = ', ssim)
print('ssim_fp = ', ssim_fp)
print('ssim_fp_old = ', ssim_fp_old)

ssim =  0.9999992221197553
ssim_fp =  0.9999961340087538
ssim_fp_old =  0.9999999540951813


In [377]:
t_orig = t_col["T"].sel(collection="orig").isel(time=0).compute()
t_zfp = t_col["T"].sel(collection="comp").isel(time=0).compute()

t_dm = ldcpy.metrics.DiffMetrics(t_orig, t_zfp, aggregate_dims=['lat', 'lon'])
t_ssim = t_dm.get_diff_metric('ssim')
t_ssim_fp = t_dm.get_diff_metric('ssim_fp')
t_ssim_fp_old = t_dm.get_diff_metric('ssim_fp_old')

In [378]:
print('ssim = ', t_ssim)
print('ssim_fp = ', t_ssim_fp)
print('ssim_fp_old = ', t_ssim_fp_old)

ssim =  0.9995286004372378
ssim_fp =  0.929348380004136
ssim_fp_old =  0.9998964778020601


In [54]:
import numpy as np
np.max(p_orig)

<xarray.DataArray 'PRECSC' ()>
array(6.071738e-08, dtype=float32)
Coordinates:
    time        object 2006-01-01 00:00:00
    collection  <U4 'orig'

In [209]:
from skimage.metrics import structural_similarity as ssim
import skimage
import skimage.io as io
#from ldcpy.s_ssim import structural_similarity as ssim_copy

t_orig = ts_col["TS"].sel(collection="orig").isel(time=0)
t_zfp = ts_col["TS"].sel(collection="zfp1.0").isel(time=0)

dm = ldcpy.metrics.DiffMetrics(t_orig, t_zfp, aggregate_dims=['lat', 'lon'])
fp = dm.get_diff_metric('ssim_fp')
fp_old = dm.get_diff_metric('ssim_fp_old')

simg1 = skimage.io.imread('/Users/abaker/alli/compression/SSIM-newtest/ts_ssim1_zfp1.0.png')
simg2 = skimage.io.imread('/Users/abaker/alli/compression/SSIM-newtest/ts_ssim2_zfp1.0.png')
imm1 = simg1[:, :, :3]
imm2 = simg2[:, :, :3]
gray1 = skimage.color.rgb2gray(imm1)
gray2 = skimage.color.rgb2gray(imm2)

sq_simg1 = skimage.io.imread('/Users/abaker/alli/compression/SSIM-newtest/sq_ts_ssim1_zfp1.0.png')
sq_simg2 = skimage.io.imread('/Users/abaker/alli/compression/SSIM-newtest/sq_ts_ssim2_zfp1.0.png')
sq_imm1 = sq_simg1[:, :, :3]
sq_imm2 = sq_simg2[:, :, :3]
sq_gray1 = skimage.color.rgb2gray(sq_imm1)
sq_gray2 = skimage.color.rgb2gray(sq_imm2)

noc_sq_simg1 = skimage.io.imread('/Users/abaker/alli/compression/SSIM-newtest/sq_noc_ts_ssim1_zfp1.0.png')
noc_sq_simg2 = skimage.io.imread('/Users/abaker/alli/compression/SSIM-newtest/sq_noc_ts_ssim2_zfp1.0.png')
noc_sq_imm1 = noc_sq_simg1[:, :, :3]
noc_sq_imm2 = noc_sq_simg2[:, :, :3]
noc_sq_gray1 = skimage.color.rgb2gray(noc_sq_imm1)
noc_sq_gray2 = skimage.color.rgb2gray(noc_sq_imm2)

plain_sq_simg1 = skimage.io.imread('/Users/abaker/alli/compression/SSIM-newtest/plain_ts_ssim1_zfp.png')
plain_sq_simg2 = skimage.io.imread('/Users/abaker/alli/compression/SSIM-newtest/plain_ts_ssim2_zfp.png')
plain_sq_imm1 = plain_sq_simg1[:, :, :3]
plain_sq_imm2 = plain_sq_simg2[:, :, :3]
plain_sq_gray1 = skimage.color.rgb2gray(plain_sq_imm1)
plain_sq_gray2 = skimage.color.rgb2gray(plain_sq_imm2)



#orig
so = ssim(imm1, imm2, multichannel=True,gaussian_weights=True,use_sample_covariance=False)
so_c = ssim(imm1, imm2, multichannel=True,gaussian_weights=True, k1 = 1e-15, K2 = 1e-15, use_sample_covariance=False)
sog = ssim(gray1, gray2, multichannel=False, data_range=1, gaussian_weights=True,use_sample_covariance=False)
sog_c = ssim(gray1, gray2, multichannel=False, data_range=1,K1=1e-15, K2 = 1e-15, gaussian_weights=True,use_sample_covariance=False)

#so = ssim_copy(gray1, gray2, multichannel=False, data_range=1, K1= 1e-10, K2 = .03, gaussian_weights=True,use_sample_covariance=False)
#s = ssim(imm1, imm2, multichannel=True)
#s2 = ssim(imm1,imm2, multichannel=True, K1=0.00001, K2 = 0.00001)
#s3 = ssim(gray1,gray2, multichannel=False, data_range=1)
#s3 = ssim(imm1,imm2, multichannel=True,gaussian_weights=True,use_sample_covariance=False)

#sq_so = ssim_copy(sq_gray1, sq_gray2, multichannel=False, data_range=1, K1= .01, K2 = .03)
#sq_s = ssim(sq_imm1, sq_imm2, multichannel=True)

#noc_sq_so = ssim_copy(noc_sq_gray1, noc_sq_gray2, multichannel=False, data_range=1, K1= .01, K2 = .03)
#noc_sq_s = ssim(noc_sq_imm1, noc_sq_imm2, multichannel=True)

#plain_sq_so = ssim_copy(plain_sq_gray1, plain_sq_gray2, multichannel=False, data_range=1, K1= .01, K2 = .03)
#plain_sq_s = ssim(plain_sq_imm1, plain_sq_imm2, multichannel=True)




In [215]:
imm1[200,200,:]


array([245, 194, 170], dtype=uint8)

In [217]:
gray1[200,200]

0.7964984313725489

In [211]:
print(so)
print(so_c)
print(sog)
print(sog_c)


0.99844309742856
0.9367751265262337
0.9985087822708693
0.9396901050525631


In [207]:
sog

0.9396901050525631

In [203]:
print('fp-ssim=', fp)
print('fp-ssim old=', fp_old)

print('sog = ', sog)
print('so = ', so)

print('ldcpy-orig', s)
#print('s2=',s2)
print('s3=',s3)
#print('sq_so = ', sq_so)
#print('sq-ldcpy-orig', sq_s)
#print('noc sq_so = ', noc_sq_so)
#print('noc sq-ldcpy-orig', noc_sq_s)
#print('plain sq_so = ', plain_sq_so)
#print('plain sq-ldcpy-orig', plain_sq_s)

fp-ssim= 0.9957813837204557
fp-ssim old= 0.9996698537643849
sog =  0.9985087822708693
so =  0.99844309742856
ldcpy-orig 0.9982826848592401
s3= 0.99844309742856


In [ ]:
np.shape(imm1)
imm1.min()



In [ ]:
import torch
import torchvision
import piq 
import numpy as np 
import skimage
import skimage.io as io
  
s1 = skimage.io.imread('/Users/abaker/alli/compression/SSIM-newtest/ts_ssim1_zfp1.0.png')
s2 = skimage.io.imread('/Users/abaker/alli/compression/SSIM-newtest/ts_ssim2_zfp1.0.png')
im1 = s1[:, :, :3]
im2 = s2[:, :, :3]
my_tensor1 = torchvision.transforms.ToTensor()(im1)
my_tensor2 = torchvision.transforms.ToTensor()(im2)

f = piq.ssim(my_tensor1, my_tensor1)
ssim_val = f.item()
g = piq.gmsd(my_tensor1, my_tensor2) 
gmsd_val = g.item()
h = piq.vif_p(my_tensor1, my_tensor2) 
vif_val = h.item()

In [ ]:
print('ssim=', ssim_val)
print('gmsd=', gmsd_val)
print('vif=', vif_val)

In [ ]:
from ldcpy.s_vif import vifp_mscale as vif
import skimage
import skimage.io as io
  
s1 = skimage.io.imread('/Users/abaker/alli/compression/SSIM-newtest/ts_ssim1_zfp1.0.png')
s2 = skimage.io.imread('/Users/abaker/alli/compression/SSIM-newtest/ts_ssim2_zfp1.0.png')
im1 = s1[:, :, :3]
im2 = s2[:, :, :3]
g1 = skimage.color.rgb2gray(im1)
g2 = skimage.color.rgb2gray(im2)
vif(g1, g2)

In [ ]:
from ldcpy.s_vif import vifp_mscale as vif
import skimage
import skimage.io as io
  
ss1 = skimage.io.imread('/Users/abaker/alli/compression/ssim/study-figs/TS_uncompressed.png')
ss2 = skimage.io.imread('/Users/abaker/alli/compression/ssim/study-figs/TS_speck_1.png')
sim1 = ss1[:, :, :3]
sim2 = ss2[:, :, :3]
sg1 = skimage.color.rgb2gray(sim1)
sg2 = skimage.color.rgb2gray(sim2)
vif(sg1, sg2)

In [ ]:
#rect plot
from numpy import inf
import copy

import matplotlib as mpl
import numpy as np
import xarray as xr
from cartopy import crs as ccrs
from cartopy.util import add_cyclic_point
from matplotlib import pyplot as plt

d1 = t_orig
d2 = t_zfp
lat1 = d1['lat']
lat2 = d2['lat']
cy1, lon1 = add_cyclic_point(d1, coord=d1['lon'])
cy2, lon2 = add_cyclic_point(d2, coord=d2['lon'])

no_inf_d1 = np.nan_to_num(cy1, nan=np.nan)
no_inf_d2 = np.nan_to_num(cy2, nan=np.nan)

color_min = min(
                np.min(d1.where(d1 != -inf)).values.min(),
                np.min(d2.where(d2 != -inf)).values.min(),
            )
color_max = max(
                np.max(d1.where(d1 != inf)).values.max(),
                np.max(d2.where(d2 != inf)).values.max(),
            )

fig = plt.figure(dpi=300, figsize=(9, 2.5))
    
mymap = copy.copy(plt.cm.get_cmap('coolwarm'))
mymap.set_under(color='black')
mymap.set_over(color='white')
mymap.set_bad(alpha=0)

#ax1 = plt.subplot(1, 2, 1, projection=ccrs.PlateCarree(central_longitude=0.0))
ax1 = plt.subplot(1, 2, 1)
ax1.set_facecolor('#39ff14')

ax1.pcolormesh(
                lon1,
                lat1,
                no_inf_d1,
                #transform=ccrs.PlateCarree(),
                cmap=mymap,
                vmin=color_min,
                vmax=color_max,
            )
#ax1.set_global()
#ax1.coastlines(linewidth=0.5)
ax1.axis('off')
plt.margins(0, 0)
extent1 = ax1.get_window_extent().transformed(fig.dpi_scale_trans.inverted())
ax1.imshow
plt.savefig('plain_ts_ssim1_zfp', bbox_inches=extent1, transparent=True, pad_inches=0)
ax1.axis('on')
    
#ax2 = plt.subplot(1, 2, 2, projection=ccrs.PlateCarree(central_longitude=0.0))
ax2 = plt.subplot(1, 2, 2)
ax2.set_facecolor('#39ff14')

ax2.pcolormesh(
                lon2,
                lat2,
                no_inf_d2,
                #transform=ccrs.PlateCarree(),
                cmap=mymap,
                vmin=color_min,
                vmax=color_max,
            )
#ax2.set_global()
#ax2.coastlines(linewidth=0.5)
plt.margins(0, 0)
ax2.imshow
ax2.axis('off')
extent2 = ax2.get_window_extent().transformed(fig.dpi_scale_trans.inverted())
plt.savefig('plain_ts_ssim2_zfp', bbox_inches=extent2, transparent=True, pad_inches=0)
ax2.axis('on')


# 

In [ ]:
a1_mu = (221.06144731415327 + 176.78527530997684 + 171.20295355827787)/3
print(a1_mu)
var_a1 = (978.7925222106047 + 633.8275343281653 + 672.9027381838312)/3
print(var_a1)


In [ ]:
import numpy as np
np.zeros(3)

In [ ]:
a1_R = 255
K1 = .01
K2 = .03
        
C1 = K1*K1*a1_R*a1_R
C2 = K2*K2*a1_R*a1_R
print(C1, C2)

In [ ]:
ds = prect_col.isel(time=0)
ldcpy.compare_stats(ds, "PRECT", "orig","zfp1e-11", aggregate_dims=['lat', 'lon'], significant_digits=8)


In [ ]:
from skimage.metrics import structural_similarity as ssim
import skimage
import skimage.io as io
from ldcpy.s_ssim import structural_similarity as ssim_copy

p_orig = prect_col["PRECT"].sel(collection="orig").isel(time=0)
p_zfp = prect_col["PRECT"].sel(collection="zfp1e-9").isel(time=0)

dm = ldcpy.metrics.DiffMetrics(p_orig, p_zfp, aggregate_dims=['lat', 'lon'])
fp = dm.get_diff_metric('ssim_fp')
fp_old = dm.get_diff_metric('ssim_fp_old')

simg1 = skimage.io.imread('/Users/abaker/alli/compression/SSIM-newtest/prect_ssim1_zfp1e-9.png')
simg2 = skimage.io.imread('/Users/abaker/alli/compression/SSIM-newtest/prect_ssim2_zfp1e-9.png')
imm1 = simg1[:, :, :3]
imm2 = simg2[:, :, :3]
gray1 = skimage.color.rgb2gray(imm1)
gray2 = skimage.color.rgb2gray(imm2)
 
#orig
#so = ssim_copy(gray1, gray2, multichannel=False, data_range=1, K1= .01, K2 = .03)
#so = ssim_copy(gray1, gray2, multichannel=False, data_range=1, K1= .01, K2 = .03,gaussian_weights=True,use_sample_covariance=False)

#origi
s = ssim(imm1, imm2, multichannel=True)
s2 = ssim(imm1,imm2, multichannel=True, gaussian_weights=True,use_sample_covariance=False)
#s3 = ssim(imm1,imm2, multichannel=True,gaussian_weights=True,use_sample_covariance=False)


In [ ]:
#print('so', so)
print('ldcpy-orig', s)
print('s2=',s2)
#print('s3=',s3)
print('fp-ssim=', fp)
print('fp-ssim old=', fp_old)

In [ ]:
from sklearn.preprocessing import minmax_scale
p_orig = prect_col["PRECT"].sel(collection="orig").isel(time=0).compute()
p_zfp = prect_col["PRECT"].sel(collection="zfp1e-9").isel(time=0).compute()
po = p_orig.data
pz = p_zfp.data
sc_po = minmax_scale(po, feature_range=(0, 1))
sc_pz = minmax_scale(pz, feature_range=(0, 1))

In [ ]:
print(po.max())
print(po.min())
print(po.mean())
print(po.std())

In [ ]:
print(sc_po.max())
print(sc_po.min())
print(sc_po.mean())
print(sc_po.std())

In [ ]:
ma = sc_po[sc_po != 0]
ma.min()

In [ ]:
ma1 = sc_po[sc_po != 1]
ma1.max()

In [ ]:
imm1.std()/255


In [5]:
dm = ldcpy.metrics.DiffMetrics(p_orig, p_zfp, aggregate_dims=['lat', 'lon'])
fp = dm.get_diff_metric('ssim_fp2')


NameError: name 'p_orig' is not defined

In [134]:
import numpy as np
mydata = np.ones([1, 6]) * np.arange(6)
mydata2 = np.ones([1, 6]) * np.random.rand(6) *10


In [135]:
mydata

array([[0., 1., 2., 3., 4., 5.]])

In [136]:
mydata2

array([[6.96356532, 1.66942373, 9.76242432, 1.71514579, 2.44746398,
        3.53284969]])

In [139]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
sc_data = scaler.fit_transform(mydata)
#sc_data2 = scaler.fit_transform(mydata2)

In [2]:
for i in range(2,8):
    bgdir ="bg_" +str(i)

    print(bgdir)

bg_2
bg_3
bg_4
bg_5
bg_6
bg_7


In [4]:
import os
curdir = os.getcwd()

In [6]:
os.chdir("collections/")

In [8]:
os.getcwd()

'/Users/abaker/alli/code-repo/ldcpy/docs/source/notebooks/collections'

In [9]:
os.chdir(curdir)

In [10]:
os.getcwd()

'/Users/abaker/alli/code-repo/ldcpy/docs/source/notebooks'

In [16]:
filename = "b.e11.BRCP85C5CNBDRD.f09_g16.031.cam.h0.CCN3.200601-201012.nc"
loc1 = filename.find('.h')


In [23]:
loc2 = filename.find('.', loc1+1)
loc2

39

In [22]:
loc3 = filename.find('.', loc2+1)
loc3

44

In [25]:
filename[loc2+1:loc3]

'CCN3'

In [1]:
from netCDF4 import Dataset
fn = "../../../data/cam-fv/orig.TS.100days.nc"
fid = Dataset(fn, "r")

In [2]:
attrs = fid.ncattrs()

In [12]:
variables = fid.variables

In [13]:
dims = fid.dimensions

In [14]:
dims["time"].size

100

In [20]:
variables["TS"].datatype

dtype('float32')

In [16]:
variables["TS"].ncattrs()


['units', 'long_name', 'cell_methods']

In [19]:
for k in variables:
    print(k)

P0
TS
gw
hyai
hyam
hybi
hybm
ilev
lat
lev
lon
time
time_bnds


In [21]:
mvarsTimeInvarient =  ["lev", "hyam", "hybm", "ilev", "hyai", "hybi", "P0", "lat",
                  "lon", "slat", "slon", "w_stag", "ntrm", "ntrn", "ntrk",
                  "ndbase", "nsbase", "nbdate", "nbsec", "mdt", "nlon",
                  "wnummax", "gw"]

In [23]:
for mk,mv in variables.items():
    print(mk)
    print(mv)

P0
<class 'netCDF4._netCDF4.Variable'>
float64 P0()
    long_name: reference pressure
    units: Pa
unlimited dimensions: 
current shape = ()
filling on, default _FillValue of 9.969209968386869e+36 used
TS
<class 'netCDF4._netCDF4.Variable'>
float32 TS(time, lat, lon)
    units: K
    long_name: Surface temperature (radiative)
    cell_methods: time: mean
unlimited dimensions: time
current shape = (100, 192, 288)
filling on, default _FillValue of 9.969209968386869e+36 used
gw
<class 'netCDF4._netCDF4.Variable'>
float64 gw(lat)
    long_name: gauss weights
unlimited dimensions: 
current shape = (192,)
filling on, default _FillValue of 9.969209968386869e+36 used
hyai
<class 'netCDF4._netCDF4.Variable'>
float64 hyai(ilev)
    long_name: hybrid A coefficient at layer interfaces
unlimited dimensions: 
current shape = (31,)
filling on, default _FillValue of 9.969209968386869e+36 used
hyam
<class 'netCDF4._netCDF4.Variable'>
float64 hyam(lev)
    long_name: hybrid A coefficient at layer midpo

In [58]:
import numpy as np
c = variables["lat"]
d = variables["gw"]
print(c)
#np.asarray(c)
print(d)

<class 'netCDF4._netCDF4.Variable'>
float64 lat(lat)
    long_name: latitude
    units: degrees_north
unlimited dimensions: 
current shape = (192,)
filling on, default _FillValue of 9.969209968386869e+36 used
<class 'netCDF4._netCDF4.Variable'>
float64 gw(lat)
    long_name: gauss weights
unlimited dimensions: 
current shape = (192,)
filling on, default _FillValue of 9.969209968386869e+36 used


In [63]:
variables["lat"] = d[:]

In [65]:
variables["lat"]

masked_array(data=[3.38174282e-05, 2.70525702e-04, 5.40978217e-04,
                   8.11284379e-04, 1.08137106e-03, 1.35116519e-03,
                   1.62059379e-03, 1.88958396e-03, 2.15806293e-03,
                   2.42595807e-03, 2.69319690e-03, 2.95970713e-03,
                   3.22541665e-03, 3.49025359e-03, 3.75414629e-03,
                   4.01702337e-03, 4.27881370e-03, 4.53944646e-03,
                   4.79885114e-03, 5.05695757e-03, 5.31369591e-03,
                   5.56899671e-03, 5.82279091e-03, 6.07500984e-03,
                   6.32558526e-03, 6.57444940e-03, 6.82153492e-03,
                   7.06677497e-03, 7.31010322e-03, 7.55145382e-03,
                   7.79076150e-03, 8.02796150e-03, 8.26298966e-03,
                   8.49578240e-03, 8.72627673e-03, 8.95441029e-03,
                   9.18012138e-03, 9.40334892e-03, 9.62403253e-03,
                   9.84211251e-03, 1.00575298e-02, 1.02702263e-02,
                   1.04801442e-02, 1.06872270e-02, 1.08914184e

In [79]:
fid.getncattr(p)

'CF-1.0'

In [80]:
for n in attrs:
    print(n, fid.getncattr(n))

Conventions CF-1.0
source CAM
case b.e11.B20TRC5CNBDRD.f09_g16.030
title UNSET
logname mai
host ys0110
Version $Name$
revision_Id $Id$
initial_file b.e11.B20TRC5CNBDRD.f09_g16.001.cam.i.1920-01-01-00000.nc
topography_file /glade/p/cesmdata/cseg/inputdata/atm/cam/topo/USGS-gtopo30_0.9x1.25_remap_c051027.nc
history Mon Jul 13 16:26:09 2020: ncks -4 -L 2 orig.TS.100days.nc orig.TS.100days.nc
Wed Oct 16 12:10:39 2019: ncks -d time,0,99,1 -v TS,gw,lat,lon,lev b.e11.B20TRC5CNBDRD.f09_g16.030.cam.h1.TS.19200101-20051231.nc orig.TS.100days.nc
Wed Oct 29 04:21:35 2014: ncks -F -d time,1,31390 b.e11.B20TRC5CNBDRD.f09_g16.030.cam.h1.TS.19200101-20051231.nc S_b.e11.B20TRC5CNBDRD.f09_g16.030.cam.h1.TS.19200101-20051231.nc
NCO netCDF Operators version 4.7.9 (Homepage = http://nco.sf.net, Code = http://github.com/nco/nco)


In [87]:
vars[0]

KeyError: 0

In [96]:
k = vars["TS"][:]

In [100]:
cfile = '/glade/work/abaker/mygit/PyCompress/sz2_1_12.config'
dbfile = cfile.encode('utf-8')

In [106]:
cfile

'/glade/work/abaker/mygit/PyCompress/sz2_1_12.config'

In [107]:
rel_tol = [.1, .05, .01, .005, .001, .0005, .0001, .00005, .00001]
for i in rel_tol:
    print(i)

0.1
0.05
0.01
0.005
0.001
0.0005
0.0001
5e-05
1e-05


In [112]:
print(filename)
loc = filename.find('.h1')
loc

b.e11.BRCP85C5CNBDRD.f09_g16.031.cam.h0.CCN3.200601-201012.nc


-1